# Tugas (LSA Topic Modelling)

## Read Data

In [9]:
#install package
!pip install sastrawi
!pip install swifter
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
#import library
import pandas as pd
from googleapiclient.discovery import build
import numpy as np
from string import punctuation
import re
import nltk

In [11]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Data didapatkan dari proses scrapping data youtube


proses scrapping dapat dilihat pada halaman berikut [scrapping_comment_yt](https://hamedayani114.github.io/prosaindata/scrapping_comment_yt.html)

In [12]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/triasmidwi/dataset/main/youtube_comment.csv')
df

,publishedAt,authorDisplayName,text,likeCount,review_token,comment_normalize,stopwords_removal,stemming
0,2023-05-11T01:22:44Z,NINIEN S,hindari jauh penggemar video,0,"['hindari', 'jauh', 'penggemar', 'video']","['hindari', 'jauh', 'penggemar', 'video']","['hindari', 'penggemar', 'video']","['hindar', 'gemar', 'video']"
1,2023-05-11T01:19:30Z,Nasrun SMD,bagi saya cuman ada calon ofsi yang bisa di pi...,0,"['bagi', 'saya', 'cuman', 'ada', 'calon', 'ofs...","['bagi', 'saya', 'cuman', 'ada', 'calon', 'ofs...","['cuman', 'calon', 'ofsi', 'pilih', 'prabowo',...","['cuman', 'calon', 'ofsi', 'pilih', 'prabowo',..."
2,2023-05-11T00:00:41Z,Zaen Arif,konten pkokkkkkk,0,"['konten', 'pkokkkkkk']","['konten', 'pkokkkkkk']","['konten', 'pkokkkkkk']","['konten', 'pkokkkkkk']"
3,2023-05-10T23:32:22Z,Ama agus Kudrotilah,wajah koruptor mnyesakan dada kalo mbunuh di...,0,"['wajah', 'koruptor', 'mnyesakan', 'dada', 'ka...","['wajah', 'koruptor', 'mnyesakan', 'dada', 'ka...","['wajah', 'koruptor', 'mnyesakan', 'dada', 'ka...","['wajah', 'koruptor', 'mnyesakan', 'dada', 'ka..."
4,2023-05-10T23:28:28Z,Ama agus Kudrotilah,kasihan lambang islam dukung ci porno yg tiada...,0,"['kasihan', 'lambang', 'islam', 'dukung', 'ci'...","['kasihan', 'lambang', 'islam', 'dukung', 'ci'...","['kasihan', 'lambang', 'islam', 'dukung', 'ci'...","['kasihan', 'lambang', 'islam', 'dukung', 'ci'..."
...,...,...,...,...,...,...,...,...
1699,2023-05-10T14:42:51Z,Seraphine Tan,bacot,0,['bacot'],['bacot'],['bacot'],['bacot']
1700,2023-05-10T13:37:23Z,Andre Cebong,tepat skali,0,"['tepat', 'skali']","['tepat', 'skali']",['skali'],['skali']
1701,2023-05-10T13:28:35Z,supardi pardi,dan apakan anda lebih mudeng dan apakah ...,0,"['dan', 'apakan', 'anda', 'lebih', 'mudeng', '...","['dan', 'apakan', 'anda', 'lebih', 'mudeng', '...","['apakan', 'mudeng', 'tau', 'arti', 'politik']","['apa', 'mudeng', 'tau', 'arti', 'politik']"
1702,2023-05-10T13:21:39Z,Ari Torong,begono bro yen kowe arep ng semarang pilih ml...,0,"['begono', 'bro', 'yen', 'kowe', 'arep', 'ng',...","['begono', 'bro', 'yen', 'kowe', 'arep', 'ng',...","['begono', 'bro', 'yen', 'kowe', 'arep', 'ng',...","['begono', 'bro', 'yen', 'kowe', 'arep', 'ng',..."


## Feature Extraction (TF-IDF)

Algoritma TF-IDF (Term Frequency – Inverse Document Frequency) adalah salah satu algoritma yang dapat digunakan untuk menganalisa hubungan antara sebuah frase/kalimat dengan sekumpulan dokumen. Contoh yang dibahas kali ini adalah mengenai penentuan urutan peringkat data berdasarkan query yang digunakan.
Inti utama dari algoritma ini adalah melakukan perhitungan nilai TF dan nilai IDF dari sebuah setiap kata kunci terhadap masing-masing dokumen

In [13]:
def joinkata(data):
  kalimat = ""
  for i in data:
    kalimat += i
    kalimat += " "
  return kalimat

text = df['stemming'].swifter.apply(joinkata)
text

AttributeError: ignored

In [ ]:
# Vectorize document using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1)
                        )

# Fit and Transform the documents
tfidf_matrix = vectorizer.fit_transform(text)

In [ ]:
print(tfidf_matrix)

## Modelling

In [ ]:
import nltk
nltk.download('stopwords', quiet=True)

In [ ]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['comment (clean)'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

### bobot kata terhadap masing masing topik

In [ ]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

### bobot setiap topik terhadap dokumen

In [ ]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["comment (clean)"], df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

In [ ]:
df_lsa['Topik'].value_counts()
